In [ ]:
import omnipy as om
from omnipy import Dataset, Model
om.setup_jupyter_ui()

In [ ]:
om.runtime.config.data.ui.jupyter.color.style = om.DarkHighContrastTintedThemingBase16ColorStyles.CHINOISERIE_MORANDI_T16
om.runtime.config.data.ui.jupyter.color.solid_background = False
om.runtime.config.root_log.log_to_stdout = False
om.runtime.config

In [ ]:
from nobel_api_output_model import NobelPrizesResult, LaureateBasic, NobelPrize

# Omnipy tasks
@om.TaskTemplate()
def create_nobel_prize_urls(category: str | None, page_size: int, num_pages: int) -> om.HttpUrlDataset:
    def _nobel_price_api_urls(category: str | None, offset: int, limit: int, ) -> om.HttpUrlModel:
        url = om.HttpUrlModel('https://api.nobelprize.org')
        url.path // '2.1' // 'nobelPrizes'
        if category:
            url.query['nobelPrizeCategory'] = category
        url.query['offset'] = offset
        url.query['limit'] = limit
        return url

    return om.HttpUrlDataset({
        page_offset: _nobel_price_api_urls(category=category, offset=page_offset, limit=page_size)
        for page_offset in range(0, num_pages * page_size, page_size)
    })

@om.TaskTemplate()
async def fetch_and_extract_laureates_per_year(nobel_prize_urls: om.HttpUrlDataset) -> Dataset[Model[list[LaureateBasic]]]:
    per_page_nobel_prize_info = await Dataset[Model[NobelPrizesResult]].load(nobel_prize_urls)
    
    per_year_nobel_prizes = Dataset[Model[list[LaureateBasic]]]()

    all_nobel_prizes = []
    for page in per_page_nobel_prize_info.values():
        all_nobel_prizes += page.nobelPrizes
        
    for entry in all_nobel_prizes:
        if entry.laureates:
            per_year_nobel_prizes[str(entry.awardYear)] = entry.laureates
    
    return per_year_nobel_prizes


# Omnipy dataflows
@om.LinearFlowTemplate(
    create_nobel_prize_urls,
    fetch_and_extract_laureates_per_year,
)
def download_and_wrangle_nobel_prizes(category: str | None, page_size: int, num_pages: int):
    ...

# Run the dataflow and show results
literature_prizes = await download_and_wrangle_nobel_prizes.run(category='lit', page_size=25, num_pages=6)
literature_prizes

In [ ]:
literature_prizes[-3:].peek(style='rose-pine-dawn-t16', bg=True, syntax='json', h_overflow='word_wrap')

In [ ]:
(await om.JsonDataset.load(["https://api.nobelprize.org/2/laureate/1032"])).full()